In [1]:
# loading packages
import pandas as pd
import numpy as np
from hyperopt import fmin

from sklearn.model_selection import KFold, cross_val_score, train_test_split 

In [2]:
# loading data
df = pd.read_csv("heart.csv", sep=',')

In [ ]:
# checking for null
df.isnull().sum(axis = 0)

In [3]:
# split
X = df.drop("target",axis=1)
y = df["target"].values


In [6]:
# create dummies
#TODO: Try this way
X = pd.get_dummies(X, columns = ['cp','thal','slope'])

In [9]:
X.head(10)

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,ca,...,cp_1,cp_2,cp_3,thal_0,thal_1,thal_2,thal_3,slope_0,slope_1,slope_2
0,63,1,145,233,1,0,150,0,2.3,0,...,0,0,1,0,1,0,0,1,0,0
1,37,1,130,250,0,1,187,0,3.5,0,...,0,1,0,0,0,1,0,1,0,0
2,41,0,130,204,0,0,172,0,1.4,0,...,1,0,0,0,0,1,0,0,0,1
3,56,1,120,236,0,1,178,0,0.8,0,...,1,0,0,0,0,1,0,0,0,1
4,57,0,120,354,0,1,163,1,0.6,0,...,0,0,0,0,0,1,0,0,0,1
5,57,1,140,192,0,1,148,0,0.4,0,...,0,0,0,0,1,0,0,0,1,0
6,56,0,140,294,0,0,153,0,1.3,0,...,1,0,0,0,0,1,0,0,1,0
7,44,1,120,263,0,1,173,0,0.0,0,...,1,0,0,0,0,0,1,0,0,1
8,52,1,172,199,1,1,162,0,0.5,0,...,0,1,0,0,0,0,1,0,0,1
9,57,1,150,168,0,1,174,0,1.6,0,...,0,1,0,0,0,1,0,0,0,1


In [19]:
#Scaling for SVM
#TODO:
#check if all columns should be scalled
 
from sklearn import preprocessing
standard_scaler = preprocessing.StandardScaler()
X_standard = standard_scaler.fit_transform(X)

In [ ]:
#train test split 
X_train, X_test, y_train, y_test = train_test_split(X_standard, y, test_size = 0.20, random_state=3)

In [ ]:
#TODO
# Cross validation

from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train.T, y_train.T)

acc = svm.score(x_test.T,y_test.T)*100
accuracies['SVM'] = acc
print("Test Accuracy of SVM Algorithm: {:.2f}%".format(acc))